In [2]:
import pandas as pd
import re

In [8]:
df =  pd.read_csv("C:/Users/FPTSHOP/OneDrive/Documents/SAV/RawData/mobileData/Dev.csv", index_col = 0)
df.shape

(2220, 4)

In [10]:
for index, row in df.iterrows():
    if row['n_star'] == 1 or row['n_star'] ==2:
        df.at[index, 'label'] = df.at[index, 'label'].replace('OTHERS', 'OTHERS#Negative')
    elif row['n_star'] == 3:
        df.at[index, 'label'] = df.at[index, 'label'].replace('OTHERS', 'OTHERS#Neutral')
    elif row['n_star'] == 5 or row['n_star'] ==4:
        df.at[index, 'label'] = df.at[index, 'label'].replace('OTHERS', 'OTHERS#Positive')

df.head()


,comment,n_star,date_time,label
index,,,,
0,"Máy ok, k chê điểm nào, chính thức là fans ViV...",5,9/7/2020,{GENERAL#Positive};{SER&ACC#Positive};
1,"Mới sài được 3 ngày sản phẩm thì khá là ok , l...",5,4/5/2020,{CAMERA#Positive};{FEATURES#Negative};{BATTERY...
2,Dùng tốt chơi game bao mượt không nóng như lời...,5,14/12/2019,{BATTERY#Positive};{PERFORMANCE#Positive};{GEN...
3,"Có ai giống mh kg, mh mua ip 7 plus , mấy bị h...",3,7/3/2019,{DESIGN#Negative};{OTHERS#Neutral};
4,Ko bjk ngày mai có ra đổi đc ko đây. Mới mua h...,2,29/03/2020,{PERFORMANCE#Negative};{OTHERS#Negative};


In [11]:
import pandas as pd

def convert_to_aspect_df(df, column_name):
    # Định nghĩa mã hóa cho cảm xúc
    sentiment_encoding = {"Negative": 1, "Neutral": 2, "Positive": 3}
    
    # Thứ tự cột mong muốn
    desired_order = ["CAMERA", "SCREEN", "GENERAL", "STORAGE", "PERFORMANCE", "SERACC", "BATTERY", "PRICE", "DESIGN", "FEATURES","OTHERS"]
    
    # Chuẩn bị dữ liệu cho DataFrame mới
    converted_data = []
    
    # Duyệt qua mỗi hàng trong DataFrame để chuyển đổi dữ liệu
    for _, row in df.iterrows():
        aspects_str = row[column_name].replace("{", "").replace("}", "")
        aspects = [aspect.split("#") for aspect in aspects_str.split(";") if aspect]

        # Bản ghi tạm thời cho mỗi hàng, khởi tạo với giá trị 0 cho mỗi khía cạnh mong muốn
        record = dict.fromkeys(desired_order, 0)  # Sử dụng thứ tự mong muốn để khởi tạo
        
        # Cập nhật giá trị cho mỗi khía cạnh dựa trên dữ liệu
        for aspect in aspects:
            aspect_name = aspect[0]  # Tên khía cạnh
            aspect_sentiment = aspect[1]  # Cảm xúc của khía cạnh
            if aspect_name in record:  # Chỉ cập nhật nếu khía cạnh có trong thứ tự mong muốn
                record[aspect_name] = sentiment_encoding.get(aspect_sentiment, 0)
        
        converted_data.append(record)

    # Tạo một DataFrame mới từ dữ liệu đã chuyển đổi
    converted_df = pd.DataFrame(converted_data, columns=desired_order)  # Đảm bảo thứ tự cột
    
    return converted_df

# Chuyển đổi
convert_df = convert_to_aspect_df(df, "label")
convert_df.head()


,CAMERA,SCREEN,GENERAL,STORAGE,PERFORMANCE,SERACC,BATTERY,PRICE,DESIGN,FEATURES,OTHERS
0,0,0,3,0,0,0,0,0,0,0,0
1,3,0,3,0,2,0,3,0,0,1,0
2,0,0,3,0,3,0,3,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,2
4,0,0,0,0,1,0,0,0,0,0,1


In [12]:
drop_col = ['n_star', 'date_time', 'label']
df = df.drop(columns=drop_col, axis=1)
df.head()

,comment
index,
0,"Máy ok, k chê điểm nào, chính thức là fans ViV..."
1,"Mới sài được 3 ngày sản phẩm thì khá là ok , l..."
2,Dùng tốt chơi game bao mượt không nóng như lời...
3,"Có ai giống mh kg, mh mua ip 7 plus , mấy bị h..."
4,Ko bjk ngày mai có ra đổi đc ko đây. Mới mua h...


In [13]:
# Giả sử df1 và df2 là hai DataFrame bạn muốn nối với nhau
df_concatenated = pd.concat([df, convert_df],axis=1)
df_concatenated.head()


,comment,CAMERA,SCREEN,GENERAL,STORAGE,PERFORMANCE,SERACC,BATTERY,PRICE,DESIGN,FEATURES,OTHERS
0,"Máy ok, k chê điểm nào, chính thức là fans ViV...",0,0,3,0,0,0,0,0,0,0,0
1,"Mới sài được 3 ngày sản phẩm thì khá là ok , l...",3,0,3,0,2,0,3,0,0,1,0
2,Dùng tốt chơi game bao mượt không nóng như lời...,0,0,3,0,3,0,3,0,0,0,0
3,"Có ai giống mh kg, mh mua ip 7 plus , mấy bị h...",0,0,0,0,0,0,0,0,1,0,2
4,Ko bjk ngày mai có ra đổi đc ko đây. Mới mua h...,0,0,0,0,1,0,0,0,0,0,1


In [14]:
df_concatenated.to_csv("./data/Val_v3.csv", index = False)